# MFEGSN — Pipeline Colab (Marker + LangExtract)

Notebook optimisé pour une exécution **pas à pas** sur Google Colab.
Il sépare clairement les **blocs texte** (explications) et les **blocs code** (exécution).

**Ordre recommandé :** Étapes 1 → 10, avec tests optionnels si besoin.

## Étape 1 — Installer les dépendances
- Systèmes :  zstd (requis pour Ollama).
- Python : marker-pdf, langextract, pillow.

In [1]:
# Dépendances système
!apt-get update -qq
!apt-get install -y zstd -qq

# Dépendances Python
!python -m pip install -q --upgrade pip
!python -m pip install -q marker-pdf[full] langextract google-generativeai pillow

print("✅ Dépendances installées.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Dépendances installées.


## Étape 2 — Monter Google Drive
Exécutez cette cellule si vos PDF sont sur Drive.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Étape 3 — Configurer les dossiers
Modifiez les chemins ci-dessous selon votre Drive.

**Note:** Chaque PDF sera traité dans son propre dossier contenant les sous-dossiers `_ANALYSES`, `_FIGURES`, `_LOGS`, `_REFERENCES`.

In [3]:
from pathlib import Path

# === MODIFIEZ CES CHEMINS ===
INPUT_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF")
OUTPUT_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD")

assert INPUT_DIR.exists(), f"❌ Dossier introuvable : {INPUT_DIR}"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Dossier global pour les logs du pipeline
GLOBAL_LOGS_DIR = OUTPUT_DIR / "_PIPELINE_LOGS"
GLOBAL_LOGS_DIR.mkdir(parents=True, exist_ok=True)

def get_doc_folders(doc_name):
    """Crée et retourne les dossiers pour un document donné."""
    doc_dir = OUTPUT_DIR / doc_name
    doc_dir.mkdir(parents=True, exist_ok=True)

    folders = {
        "root": doc_dir,
        "figures": doc_dir / "_FIGURES",
        "references": doc_dir / "_REFERENCES",
        "analyses": doc_dir / "_ANALYSES",
        "logs": doc_dir / "_LOGS",
    }

    for folder in folders.values():
        folder.mkdir(parents=True, exist_ok=True)

    return folders

pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix. lower() == ".pdf"])
print("=" * 60)
print("📁 CONFIGURATION")
print("=" * 60)
print(f"📂 Entrée  : {INPUT_DIR}")
print(f"📂 Sortie  : {OUTPUT_DIR}")
print(f"📄 PDFs    : {len(pdf_files)}")
print(f"📂 Structure par fichier :  <nom_fichier>/")
print(f"   ├── _ANALYSES/")
print(f"   ├── _FIGURES/")
print(f"   ├── _LOGS/")
print(f"   └── _REFERENCES/")
print("=" * 60)

📁 CONFIGURATION
📂 Entrée  : /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF
📂 Sortie  : /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD
📄 PDFs    : 54
📂 Structure par fichier :  <nom_fichier>/
   ├── _ANALYSES/
   ├── _FIGURES/
   ├── _LOGS/
   └── _REFERENCES/


## Étape 4 — Ollama + Gemma 3 4B
Configuration par défaut :  Ollama est **activé** pour utiliser Gemma localement.

In [6]:
USE_OLLAMA = True  # Activé par défaut pour Gemma local

ollama_process = None

if USE_OLLAMA:
    import subprocess
    import time
    import os
    import requests

    # Installer Ollama (si besoin)
    print("📦 Installation d'Ollama...")
    !curl -fsSL https://ollama.com/install.sh 2>/dev/null | sh 2>&1 | tail -n 3

    # Démarrer Ollama en arrière-plan
    ollama_process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    print("⏳ Démarrage du serveur Ollama...")

    # Attendre que le serveur soit prêt
    for i in range(30):
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=2)
            if response.status_code == 200:
                print("✅ Serveur Ollama prêt!")
                break
        except:
            pass
        time.sleep(1)
    else:
        print("⚠️ Le serveur Ollama n'a pas démarré à temps")

    # Télécharger Gemma 3 4B (sans espace avant 4b!)
    print("📥 Téléchargement de Gemma 3 4B (≈3GB, peut prendre plusieurs minutes)...")
    !ollama pull gemma3:4b

    # Vérification
    result = subprocess.run(["ollama", "list"], capture_output=True, text=True)
    if "gemma3:4b" in result.stdout:
        print("✅ Gemma 3 4B téléchargé!")

        # Lancer/préchauffer le modèle
        print("🚀 Démarrage de Gemma 3 4B...")
        !ollama run gemma3:4b "Réponds uniquement:  OK" --verbose 2>&1 | head -5
        print("✅ Gemma 3 4B prêt et opérationnel!")
    else:
        print("❌ Erreur:  Gemma 3 4B non trouvé")
        print("Modèles disponibles:")
        print(result.stdout if result.stdout else "Aucun modèle")
else:
    print("ℹ️ Ollama désactivé.  Mettez USE_OLLAMA = True pour l'activer.")

📦 Installation d'Ollama...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
⏳ Démarrage du serveur Ollama...
✅ Serveur Ollama prêt!
📥 Téléchargement de Gemma 3 4B (≈3GB, peut prendre plusieurs minutes)...

✅ Gemma 3 4B téléchargé!
🚀 Démarrage de Gemma 3 4B...
⠙ ⠹ ⠹ ⠼ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠋ ⠙ ⠹ ⠼ ⠴ ⠦ ⠧ ⠧ ⠇ ⠋ ⠙ ⠙ ⠹ ⠼ ⠼ ⠦ ⠦ ⠇ ⠏ ⠏ ⠋ ⠹ ⠹ ⠼ ⠼ ⠦ ⠦ ⠧ ⠏ ⠏ ⠙ ⠹ ⠹ ⠸ ⠼ ⠦ ⠧ ⠧ ⠏ ⠏ ⠙ ⠙ ⠸ ⠼ ⠼ ⠴ ⠦ ⠧ ⠏ ⠏ ⠙ ⠙ ⠸ ⠼ ⠼ ⠴ OK


total duration:       6.700090953s
load duration:        6.512741475s
✅ Gemma 3 4B prêt et opérationnel!


## Étape 5 — Configurer Marker
Réglages optimisés (2 workers + extraction figures + références).

**Note:** Les logs sont supprimés pour éviter les ralentissements du navigateur.

In [ ]:
import json
import re
import shutil
import base64
import logging
import warnings
import os
from datetime import datetime
from contextlib import redirect_stdout, redirect_stderr
from io import StringIO

# === SUPPRESSION DES LOGS VERBEUX ===
# Désactiver les warnings
warnings.filterwarnings('ignore')

# Configurer le logging au niveau ERROR uniquement
logging.getLogger().setLevel(logging.ERROR)
for logger_name in ['transformers', 'torch', 'marker', 'PIL', 'urllib3', 'filelock', 'huggingface_hub']:
    logging. getLogger(logger_name).setLevel(logging.ERROR)

# Variables d'environnement pour réduire les logs
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import torch
from PIL import Image

device = "cuda" if torch. cuda.is_available() else "cpu"
print(f"🖥️ Device:  {device}")
if device == "cuda":
    print(f"   GPU:  {torch.cuda. get_device_name(0)}")
    print(f"   VRAM: {torch. cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Configuration Marker avec Gemma local par défaut
marker_config = {
    "workers": 2,
    "extract_images": True,
    "images_as_base64": False,
    "use_llm": USE_OLLAMA,
    "llm_provider": "ollama" if USE_OLLAMA else None,
    "llm_model":  "gemma3:4b" if USE_OLLAMA else None,
    "force_ocr": False,
    "languages": ["fr", "en"],
    "paginate_output": True,
    "batch_size": 4 if device == "cuda" else 2,
}

print("📥 Chargement des modèles Marker (logs supprimés)...")
print(f"⚙️ Configuration:  {marker_config['workers']} workers, batch_size={marker_config['batch_size']}")
if USE_OLLAMA:
    print(f"🤖 LLM:  Gemma 3 4B via Ollama")

# Charger les modèles en supprimant les sorties
from marker.converters. pdf import PdfConverter
from marker.models import create_model_dict
from marker.config. parser import ConfigParser

# Capturer et ignorer les sorties lors du chargement
_stdout_capture = StringIO()
_stderr_capture = StringIO()

with redirect_stdout(_stdout_capture), redirect_stderr(_stderr_capture):
    model_dict = create_model_dict()
    config_parser = ConfigParser(marker_config)
    converter = PdfConverter(
        config=config_parser. generate_config_dict(),
        artifact_dict=model_dict,
    )

print("✅ Marker configuré avec 2 workers! ")

🖥️ Device:  cuda
   GPU:  Tesla T4
   VRAM: 15.8 GB
📥 Chargement des modèles Marker (logs supprimés)...
⚙️ Configuration:  2 workers, batch_size=4
🤖 LLM:  Gemma 3 4B via Ollama


## Étape 6 — Fonctions utilitaires
Extraction des références, figures et conversion PDF → Markdown.

In [ ]:
def extract_references_from_markdown(markdown_text):
    """Extrait la section références/bibliographie du Markdown."""
    references = {
        "references_text": "",
        "references_list": [],
        "reference_count": 0,
    }

    ref_patterns = [
        r"(? i)(? :^|\n)#{1,3}\s*(references|références|bibliography|bibliographie|works\s*cited|sources?)\s*[\s: ]*\n([\s\S]*?)(?=\n#{1,3}\s|\Z)",
        r"(?i)(?:^|\n)\*\*(references|références|bibliography|bibliographie)\*\*\s*[\s:]*\n([\s\S]*?)(?=\n\*\*|\n#{1,3}|\Z)",
    ]

    for pattern in ref_patterns:
        match = re.search(pattern, markdown_text, re.MULTILINE)
        if match:
            ref_section = match.group(2).strip()
            references["references_text"] = ref_section

            ref_lines = []
            lines = ref_section. split("\n")
            current_ref = ""

            for line in lines:
                line = line.strip()
                if not line:
                    if current_ref:
                        ref_lines. append(current_ref. strip())
                        current_ref = ""
                    continue

                if re.match(r"^(\[\d+\]|\d+\.|[-•]|\([A-Z])", line):
                    if current_ref:
                        ref_lines.append(current_ref.strip())
                    current_ref = line
                else:
                    current_ref += " " + line

            if current_ref:
                ref_lines. append(current_ref.strip())

            references["references_list"] = [r for r in ref_lines if len(r) > 20]
            references["reference_count"] = len(references["references_list"])
            break

    return references


def extract_figures_info(markdown_text, images_dict):
    """Extrait les informations sur les figures du document."""
    figures = []
    fig_pattern = r"(?i)(figure|fig\.)\s*(\d+)?\s*[:]? \s*(. {0,120})"

    for match in re.finditer(fig_pattern, markdown_text):
        title = (match.group(3) or "").strip()
        figures.append({
            "label": match.group(0).strip(),
            "title": title,
        })

    if images_dict:
        for img_name in images_dict. keys():
            existing = any(f. get("path") == img_name for f in figures)
            if not existing:
                figures.append({
                    "label": str(img_name),
                    "title":  "",
                    "path": str(img_name),
                })

    return figures


def save_figures(images_dict, doc_folders):
    """Sauvegarde les figures extraites dans le dossier _FIGURES du document."""
    if not images_dict:
        return []

    figures_folder = doc_folders["figures"]
    saved_paths = []

    for img_name, img_data in images_dict.items():
        safe_name = re.sub(r"[^a-zA-Z0-9_-]+", "_", str(img_name))
        img_path = figures_folder / f"{safe_name}.png"

        try:
            if isinstance(img_data, Image.Image):
                img_data.save(img_path)
            elif isinstance(img_data, (bytes, bytearray)):
                with open(img_path, "wb") as f:
                    f.write(img_data)
            elif isinstance(img_data, str):
                if img_data.startswith("data:image"):
                    b64_data = img_data.split(",", 1)[1]
                    with open(img_path, "wb") as f:
                        f. write(base64.b64decode(b64_data))
                elif Path(img_data).exists():
                    shutil.copy(img_data, img_path)
                else:
                    with open(img_path, "wb") as f:
                        f.write(base64.b64decode(img_data))
            else:
                continue

            saved_paths.append(str(img_path))
        except Exception:
            continue

    return saved_paths


def convert_pdf_complete(pdf_path, doc_name):
    """Conversion complète d'un PDF avec extraction figures et références."""
    doc_folders = get_doc_folders(doc_name)

    result_data = {
        "doc_name":  doc_name,
        "doc_folder": str(doc_folders["root"]),
        "markdown_path": "",
        "figures": [],
        "figures_paths": [],
        "references": {},
        "error": None,
    }

    try:
        result = converter(str(pdf_path))
        markdown_text = getattr(result, "markdown", "") or ""
        images_dict = getattr(result, "images", {}) or {}

        md_path = doc_folders["root"] / f"{doc_name}. md"
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(markdown_text)

        result_data["markdown_path"] = str(md_path)
        result_data["figures_paths"] = save_figures(images_dict, doc_folders)
        result_data["figures"] = extract_figures_info(markdown_text, images_dict)

        result_data["references"] = extract_references_from_markdown(markdown_text)
        if result_data["references"]. get("reference_count", 0) > 0:
            ref_path = doc_folders["references"] / f"{doc_name}_references. json"
            with open(ref_path, "w", encoding="utf-8") as f:
                json.dump(result_data["references"], f, ensure_ascii=False, indent=2)

        return result_data
    except Exception as e:
        result_data["error"] = str(e)
        error_log = doc_folders["logs"] / "error.txt"
        with open(error_log, "w", encoding="utf-8") as f:
            f.write(f"Erreur:  {str(e)}\nDate: {datetime.now().isoformat()}")
        return result_data

print("✅ Fonctions utilitaires définies. ")

## Étape 7 — LangExtract
Extraction structurée avec LangExtract (activé par défaut avec Ollama).

In [ ]:
USE_LANGEXTRACT = True

LANGEXTRACT_CONFIG = {
    "provider": "ollama" if USE_OLLAMA else "openai",
    "model": "gemma3:4b" if USE_OLLAMA else "gpt-3.5-turbo",
    "base_url": "http://localhost:11434" if USE_OLLAMA else None,
}

PROMPT_TEMPLATE = """Vous êtes un assistant d'analyse pour des documents en sciences sociales.
Retournez un JSON structuré avec les sections suivantes:

1.  CONTEXTE
- Thème principal
- Zone géographique
- Période

2. ACTEURS
- Institutions
- Pays
- Organisations

3. CONCEPTS CLÉS
- Mots-clés
- Concepts

4. DONNÉES
- Chiffres clés (si disponibles)

5. RÉFÉRENCES
- Principales références citées

6. FIGURES ET TABLEAUX
- Liste des figures mentionnées

Répondez uniquement avec un JSON valide."""

def _safe_json(obj):
    try:
        return json. loads(json.dumps(obj))
    except Exception:
        return {"raw": str(obj)}


def extract_with_langextract(markdown_text, doc_name, doc_folders, references_data=None, figures_data=None):
    """Extraction structurée avec LangExtract."""
    if not USE_LANGEXTRACT:
        return {"status": "skipped", "reason": "USE_LANGEXTRACT=False"}

    enriched_text = markdown_text

    if references_data and references_data.get("reference_count", 0) > 0:
        enriched_text += "\n\n## RÉFÉRENCES\n"
        enriched_text += f"Nombre de références: {references_data['reference_count']}\n"
        for i, ref in enumerate(references_data.get("references_list", [])[:20], 1):
            enriched_text += f"[{i}] {ref}\n"

    if figures_data:
        enriched_text += "\n\n## FIGURES IDENTIFIÉES\n"
        enriched_text += f"Nombre de figures: {len(figures_data)}\n"
        for fig in figures_data[: 10]:
            enriched_text += f"- {fig. get('label', '')} {fig.get('title', '')}\n"

    try:
        import langextract as lx

        extractor_kwargs = {
            "prompt": PROMPT_TEMPLATE,
        }

        if USE_OLLAMA and LANGEXTRACT_CONFIG["base_url"]:
            extractor_kwargs["provider"] = "ollama"
            extractor_kwargs["model"] = LANGEXTRACT_CONFIG["model"]
            extractor_kwargs["base_url"] = LANGEXTRACT_CONFIG["base_url"]

        if hasattr(lx, "extract"):
            extraction = lx.extract(enriched_text, **extractor_kwargs)
        elif hasattr(lx, "LangExtract"):
            extractor = lx. LangExtract(**extractor_kwargs)
            extraction = extractor.extract(enriched_text)
        else:
            return {"status":  "error", "error": "API LangExtract introuvable"}

        result = _safe_json(extraction)

        extraction_path = doc_folders["analyses"] / f"{doc_name}_langextract.json"
        with open(extraction_path, "w", encoding="utf-8") as f:
            json.dump(result, f, ensure_ascii=False, indent=2)

        return result
    except Exception as e:
        return {"status": "error", "error": str(e)}

print(f"✅ LangExtract configuré (Provider: {LANGEXTRACT_CONFIG['provider']}, Model: {LANGEXTRACT_CONFIG['model']})")

## Étape 8 — Test sur un PDF (optionnel)
Permet de valider la configuration avant le batch.

In [ ]:
pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix. lower() == ".pdf"])

if pdf_files:
    sample_path = pdf_files[0]
    sample_name = sample_path. stem
    print(f"🧪 Test sur:  {sample_path.name}")
    sample_result = convert_pdf_complete(sample_path, sample_name)

    if sample_result. get("error"):
        print(f"❌ Erreur: {sample_result['error']}")
    else:
        print(f"��� Conversion réussie! ")
        print(f"   📂 Dossier:  {sample_result['doc_folder']}")
        print(f"   📄 Markdown: {sample_result['markdown_path']}")
        print(f"   🖼️ Figures: {len(sample_result['figures_paths'])}")
        print(f"   📚 Références: {sample_result['references']. get('reference_count', 0)}")
else:
    print("❌ Aucun PDF trouvé dans le dossier d'entrée.")

## Étape 9 — Pipeline complet avec reprise
Traitement batch + logs + reprise automatique.

In [ ]:
import time as time_module

def process_all_documents():
    """Traite tous les documents PDF avec reprise automatique."""

    all_pdfs = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])

    if not all_pdfs:
        print("❌ Aucun PDF trouvé dans le dossier d'entrée! ")
        print(f"   Chemin vérifié: {INPUT_DIR}")
        return [], []

    log_file = GLOBAL_LOGS_DIR / f"processing_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    progress_file = GLOBAL_LOGS_DIR / "progress. json"

    def log_message(message):
        print(message)
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(message + "\n")

    processed_files = set()
    if progress_file.exists():
        try:
            with open(progress_file, "r", encoding="utf-8") as f:
                progress_data = json.load(f)
                processed_files = set(progress_data. get("processed", []))
            log_message(f"📂 Reprise:  {len(processed_files)} fichiers déjà traités")
        except Exception:
            processed_files = set()

    remaining_pdfs = [p for p in all_pdfs if p. name not in processed_files]

    log_message("=" * 60)
    log_message(f"📄 Total PDFs:  {len(all_pdfs)}")
    log_message(f"✅ Déjà traités:  {len(processed_files)}")
    log_message(f"⏳ Restants: {len(remaining_pdfs)}")
    log_message("=" * 60)

    if not remaining_pdfs:
        log_message("✅ Tous les fichiers ont déjà été traités!")
        return [], []

    results = []
    errors = []
    start_time = time_module.time()

    def save_progress():
        with open(progress_file, "w", encoding="utf-8") as f:
            json.dump({
                "processed": list(processed_files),
                "last_update": datetime.now().isoformat()
            }, f, ensure_ascii=False, indent=2)

    for idx, pdf_path in enumerate(remaining_pdfs, 1):
        doc_name = pdf_path.stem
        doc_start_time = time_module.time()

        log_message(f"\n[{idx}/{len(remaining_pdfs)}] 🚀 Traitement:  {pdf_path.name}")

        conversion_result = convert_pdf_complete(pdf_path, doc_name)

        if conversion_result.get("error"):
            errors.append({"file": pdf_path.name, "error": conversion_result["error"]})
            log_message(f"   ❌ Erreur conversion: {conversion_result['error']}")
            processed_files.add(pdf_path.name)
            save_progress()
            continue

        doc_folders = get_doc_folders(doc_name)

        log_message(
            f"   🖼️ Figures:  {len(conversion_result['figures'])} trouvées, "
            f"{len(conversion_result['figures_paths'])} sauvegardées"
        )
        log_message(
            f"   📚 Références: {conversion_result['references']. get('reference_count', 0)} extraites"
        )

        extraction = None
        if USE_LANGEXTRACT:
            log_message("   🔍 Extraction LangExtract... ")
            with open(conversion_result['markdown_path'], 'r', encoding='utf-8') as f:
                md_content = f.read()
            extraction = extract_with_langextract(
                md_content,
                doc_name,
                doc_folders,
                conversion_result['references'],
                conversion_result['figures'],
            )
            if extraction. get("status") == "error":
                log_message(f"   ⚠️ LangExtract:  {extraction. get('error', 'Erreur inconnue')}")
            else:
                log_message("   ✅ LangExtract terminé")

        analysis = {
            "doc_name": doc_name,
            "source_pdf": str(pdf_path),
            "doc_folder": str(doc_folders["root"]),
            "processed_at": datetime.now().isoformat(),
            "processing_time_seconds": round(time_module.time() - doc_start_time, 2),
            "conversion": {
                "markdown_path": conversion_result['markdown_path'],
                "figures_count": len(conversion_result['figures']),
                "figures_saved": len(conversion_result['figures_paths']),
                "figures_paths": conversion_result['figures_paths'],
                "references_count":  conversion_result['references'].get('reference_count', 0),
            },
            "references": conversion_result['references'],
            "figures": conversion_result['figures'],
            "langextract": extraction,
        }

        analysis_path = doc_folders["analyses"] / f"{doc_name}_analysis.json"
        with open(analysis_path, "w", encoding="utf-8") as f:
            json.dump(analysis, f, ensure_ascii=False, indent=2)

        doc_log_path = doc_folders["logs"] / "processing. log"
        with open(doc_log_path, "w", encoding="utf-8") as f:
            f. write(f"Traité le: {datetime.now().isoformat()}\n")
            f. write(f"Durée: {analysis['processing_time_seconds']} secondes\n")
            f. write(f"Figures: {len(conversion_result['figures_paths'])}\n")
            f. write(f"Références: {conversion_result['references'].get('reference_count', 0)}\n")

        results.append(analysis)
        processed_files.add(pdf_path.name)
        save_progress()

        elapsed = time_module.time() - doc_start_time
        log_message(f"   ⏱️ Durée: {elapsed:. 1f}s")

    total_time = time_module.time() - start_time
    log_message("\n" + "=" * 60)
    log_message(f"✅ Traitement terminé en {total_time:. 1f} secondes")
    log_message(f"   📄 Réussis: {len(results)}")
    log_message(f"   ❌ Erreurs: {len(errors)}")
    log_message("=" * 60)

    return results, errors


def export_all_results(results):
    """Exporte les résultats globaux."""
    if not results:
        print("ℹ️ Aucun résultat à exporter.")
        return

    summary_path = GLOBAL_LOGS_DIR / "_SUMMARY.json"
    report_md_path = GLOBAL_LOGS_DIR / "_REPORT.md"

    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    all_references = []
    for data in results:
        refs = data.get("references", {})
        if refs. get("references_list"):
            for ref in refs["references_list"]:
                all_references.append({
                    "document": data. get("doc_name", ""),
                    "reference": ref,
                })

    if all_references:
        biblio_path = GLOBAL_LOGS_DIR / "_BIBLIOGRAPHIE_COMPLETE.json"
        with open(biblio_path, "w", encoding="utf-8") as f:
            json. dump(all_references, f, ensure_ascii=False, indent=2)
        print(f"📚 Bibliographie:  {biblio_path} ({len(all_references)} références)")

    all_figures = []
    for data in results:
        for fig_path in data.get("conversion", {}).get("figures_paths", []):
            all_figures. append({
                "document": data. get("doc_name", ""),
                "path": fig_path,
            })

    if all_figures:
        figures_index_path = GLOBAL_LOGS_DIR / "_INDEX_FIGURES.json"
        with open(figures_index_path, "w", encoding="utf-8") as f:
            json.dump(all_figures, f, ensure_ascii=False, indent=2)
        print(f"🖼️ Index figures: {figures_index_path} ({len(all_figures)} figures)")

    md_lines = []
    md_lines.append("# Rapport de traitement MFEGSN\n\n")
    md_lines. append(f"**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")
    md_lines. append(f"- **Total documents traités:** {len(results)}\n")
    md_lines.append(f"- **Total figures extraites:** {len(all_figures)}\n")
    md_lines.append(f"- **Total références:** {len(all_references)}\n\n")
    md_lines.append("## Documents traités\n\n")

    for data in results:
        conv = data.get("conversion", {})
        md_lines. append(f"### {data.get('doc_name', '')}\n\n")
        md_lines. append(f"- **Dossier:** `{data.get('doc_folder', '')}`\n")
        md_lines. append(f"- **Figures:** {conv.get('figures_count', 0)}\n")
        md_lines.append(f"- **Références:** {conv.get('references_count', 0)}\n")
        md_lines.append(f"- **Durée:** {data.get('processing_time_seconds', 0)}s\n\n")

    with open(report_md_path, "w", encoding="utf-8") as f:
        f. write("".join(md_lines))

    print(f"✅ Résultats exportés: ")
    print(f"   📊 Summary: {summary_path}")
    print(f"   📝 Report: {report_md_path}")

print("✅ Fonctions de pipeline définies.")

## Étape 10 — Lancer le traitement
Exécutez cette cellule pour lancer le traitement complet.

In [ ]:
print("🚀 Démarrage du pipeline... \n")

results, errors = process_all_documents()

if results:
    export_all_results(results)

print("\n" + "=" * 60)
print("✅ PIPELINE TERMINÉ")
print("=" * 60)

if errors:
    print(f"\n⚠️ {len(errors)} erreur(s) rencontrée(s):")
    for err in errors[: 5]:
        print(f"   - {err['file']}: {err['error'][:50]}...")
    if len(errors) > 5:
        print(f"   ... et {len(errors) - 5} autres erreurs")